<a href="https://colab.research.google.com/github/JulienHelfenstein/World_model/blob/main/04_train_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Définir le chemin racine de votre projet
PROJECT_ROOT = "/content/drive/My Drive/Colab Notebooks/World_model"

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
import gymnasium
from tqdm import tqdm

In [ ]:
# --- 1. Configuration et Hyperparamètres ---
VAE_MODEL_PATH = os.path.join(PROJECT_ROOT, "vae.pth")
RNN_MODEL_PATH = os.path.join(PROJECT_ROOT, "rnn.pth")
CONTROLLER_SAVE_PATH = os.path.join(PROJECT_ROOT, "controller.pth")

# Paramètres (doivent correspondre aux autres scripts)
z_dim = 32
action_dim = 3  # CarRacing: [steer, gas, brake]
hidden_dim = 256 # Mémoire du LSTM
num_mixtures = 5

# Paramètres de l'entraînement du Contrôleur
DREAM_HORIZON = 500 # Nombre d'étapes à simuler dans le rêve
NUM_GENERATIONS = 100 # Nombre de "générations" d'entraînement

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# --- 2. Définition des Modèles (Classes) ---

# On a besoin de la définition du CVAE pour charger 'vae.pth'
# (On n'a besoin que de l'encodeur)
class CVAE(nn.Module):
    def __init__(self, z_dim, image_channels=3):
        super(CVAE, self).__init__()
        self.z_dim = z_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.ReLU()
        )
        self.flat_size = 256 * 4 * 4
        self.fc_mu = nn.Linear(self.flat_size, z_dim)
        self.fc_logvar = nn.Linear(self.flat_size, z_dim)

    def encode(self, x):
        h = self.encoder(x)
        h_flat = h.view(-1, self.flat_size)
        return self.fc_mu(h_flat), self.fc_logvar(h_flat)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var); eps = torch.randn_like(std)
        return mu + eps * std

# Définition du MDNRNN (MODIFIÉ pour inclure la récompense et le 'done')
#
class MDNRNN(nn.Module):
    def __init__(self, z_dim, action_dim, hidden_dim, num_mixtures):
        super(MDNRNN, self).__init__()
        self.z_dim = z_dim
        self.hidden_dim = hidden_dim
        self.num_mixtures = num_mixtures

        input_dim = z_dim + action_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        # Tête pour z_{t+1} (le MDN)
        mdn_output_dim = num_mixtures * (1 + 2 * z_dim)
        self.mdn_output = nn.Linear(hidden_dim, mdn_output_dim)

        # --- NOUVELLES TÊTES ---
        # Tête pour la récompense r_t (une simple valeur)
        self.reward_head = nn.Linear(hidden_dim, 1)
        # Tête pour le 'done' d_t (une probabilité)
        self.done_head = nn.Linear(hidden_dim, 1)

    def forward(self, z_t, a_t, hidden_state):
        # z_t shape: (batch_size, z_dim)
        # a_t shape: (batch_size, action_dim)
        lstm_input = torch.cat([z_t, a_t], dim=-1).unsqueeze(1) # Ajoute dim 'seq'

        lstm_output, next_hidden = self.lstm(lstm_input, hidden_state)
        lstm_output = lstm_output.squeeze(1) # Enlève dim 'seq'

        # Prédire les 3 sorties
        mdn_params = self.mdn_output(lstm_output)
        pred_reward = self.reward_head(lstm_output)
        pred_done_logits = self.done_head(lstm_output)

        return mdn_params, pred_reward, pred_done_logits, next_hidden

    def sample_z_from_mdn(self, mdn_params):
        # Fonction (simplifiée) pour piocher un z_{t+1} du mélange
        # (Dans un vrai code, ce serait la logique MDN de la Phase 2)

        # Pour cet exemple, on simplifie : on prend la moyenne (mu)
        # de la première mixture comme "prédiction"
        # (batch_size, num_mixtures * (1 + 2*z_dim))

        # Ceci est une *simplification pédagogique* !
        # Un vrai sample piocherait dans le mélange.
        first_mu_start = self.num_mixtures
        first_mu_end = self.num_mixtures + self.z_dim
        pred_z_mu = mdn_params[..., first_mu_start:first_mu_end]
        return pred_z_mu

# Définition du Contrôleur (simple, comme avant)
class Controller(nn.Module):
    def __init__(self, z_dim, hidden_dim, action_dim):
        super(Controller, self).__init__()
        input_dim = z_dim + hidden_dim
        self.fc = nn.Linear(input_dim, action_dim)

    def forward(self, z_t, h_t):
        controller_input = torch.cat([z_t, h_t], dim=-1)
        action_unscaled = self.fc(controller_input)

        # Actions pour CarRacing: [steer, gas, brake]
        # On utilise tanh pour le volant (-1, 1)
        # On utilise sigmoid pour l'accélérateur et le frein (0, 1)
        steer = torch.tanh(action_unscaled[:, 0:1])
        gas = torch.sigmoid(action_unscaled[:, 1:2])
        brake = torch.sigmoid(action_unscaled[:, 2:3])

        return torch.cat([steer, gas, brake], dim=-1)

In [ ]:
# --- 3. La Fonction de "Rêve" (le Cœur) ---
def simulate_dream(controller, vae, rnn, z_0, h_0):
    """
    Simule une trajectoire complète dans le rêve et retourne la récompense totale.
    """
    controller.eval(); vae.eval(); rnn.eval() # Mode évaluation

    total_reward = 0
    z_t = z_0
    h_t, c_t = h_0

    with torch.no_grad():
        for _ in range(DREAM_HORIZON):
            # 1. Le Contrôleur décide d'une action
            a_t = controller(z_t, h_t)

            # 2. Le RNN (Moteur de Rêve) prédit le futur
            mdn_params, pred_reward, pred_done_logits, (h_t, c_t) = rnn(z_t, a_t, (h_t, c_t))

            # 3. On "pioch" le prochain z_{t+1}
            z_t = rnn.sample_z_from_mdn(mdn_params)

            # 4. On accumule la récompense prédite
            total_reward += pred_reward.item()

            # 5. On vérifie si le rêve est "terminé"
            if torch.sigmoid(pred_done_logits).item() > 0.5:
                break # L'agent a "rêvé" qu'il crashait

    return total_reward

In [ ]:
# --- 4. Script Principal d'Entraînement (Phase 3) ---
if __name__ == "__main__":

    # 1. Charger les modèles VAE et RNN
    print("Chargement des modèles VAE et RNN...")
    vae = CVAE(z_dim).to(device)
    vae.load_state_dict(torch.load(VAE_MODEL_PATH, map_location=device))

    rnn = MDNRNN(z_dim, action_dim, hidden_dim, num_mixtures).to(device)
    rnn.load_state_dict(torch.load(RNN_MODEL_PATH, map_location=device))
    print("Modèles chargés.")

    # 2. Initialiser le Contrôleur
    controller = Controller(z_dim, hidden_dim, action_dim).to(device)

    # 3. Obtenir un "point de départ" (z_0, h_0) du VRAI monde
    # (C'est la seule fois qu'on touche à l'environnement)
    print("Obtention d'un état de départ (z_0) du vrai monde...")
    env = gymnasium.make('CarRacing-v2')
    obs, _ = env.reset()
    obs_img = (torch.from_numpy(obs).permute(2, 0, 1).float() / 255.0).unsqueeze(0).to(device)
    # Note: L'image doit être redimensionnée à 64x64 comme pour le VAE
    # Ici, nous allons tricher et supposer que obs_img est 64x64
    # (dans un vrai code, il faut importer la fct de redimensionnement)

    with torch.no_grad():
        # L'image est 96x96, le VAE s'attend à 64x64. On redimensionne.
        obs_img_64 = F.interpolate(obs_img, size=(64, 64), mode='bilinear', align_corners=False)
        mu, log_var = vae.encode(obs_img_64)
        z_0 = vae.reparameterize(mu, log_var)

    h_0 = (torch.zeros(1, 1, hidden_dim).to(device),
           torch.zeros(1, 1, hidden_dim).to(device))
    env.close()

    # 4. BOUCLE D'ENTRAÎNEMENT (CONCEPTUELLE)
    # L'entraînement du Contrôleur n'est pas fait par backpropagation.
    # On utilise un algorithme "boîte noire" (black-box),
    # comme un Algorithme Génétique ou CMA-ES.

    print("Début de l'entraînement 'boîte noire' du Contrôleur (conceptuel)...")

    # Ceci est une FAUSSE boucle d'optimisation pour l'exemple.
    # Dans la vraie vie, vous utiliseriez une bibliothèque comme `cma`.

    # On utilise Adam comme un "faux" optimiseur d'évolution
    # C'est une astuce, pas la méthode standard, mais ça illustre l'idée
    optimizer = torch.optim.Adam(controller.parameters(), lr=1e-3)

    for generation in range(NUM_GENERATIONS):
        controller.train()

        # On simule le rêve
        total_reward = simulate_dream(controller, vae, rnn, z_0, h_0)

        # Puisqu'on veut MAXIMISER la récompense, on minimise son OPPOSÉ
        loss = -total_reward

        # On fait une passe "backward" (ceci est une astuce qui
        # ne fonctionne qu'avec certains types de RL, mais
        # illustre la mise à jour)
        # Note: Pour que cela fonctionne, 'simulate_dream' ne devrait
        # pas avoir @torch.no_grad() et les .item().

        # ---- VRAIE FAÇON (Conceptuelle) ----
        # 1. best_reward = -infinity
        # 2. for i in range(50): # 50 agents par génération
        # 3.   créer un 'new_controller' avec des poids bruités
        # 4.   reward = simulate_dream(new_controller, ...)
        # 5.   si reward > best_reward, garder ce contrôleur
        # 6. 'controller' = meilleur contrôleur de la génération
        # -----------------------------------

        # On va juste afficher la récompense pour l'exemple
        print(f"Génération {generation+1}/{NUM_GENERATIONS}, Récompense de Rêve: {total_reward:.2f}")

        # (Ici, on ne met pas vraiment le contrôleur à jour,
        # car il manque l'algorithme d'évolution)

    # 5. Sauvegarder le meilleur contrôleur
    print("Entraînement terminé.")
    torch.save(controller.state_dict(), CONTROLLER_SAVE_PATH)
    print(f"Modèle Contrôleur sauvegardé dans {CONTROLLER_SAVE_PATH}")